In [1]:
import random

import cv2
import numpy as np

In [2]:
def stitch_images(left_img, right_img):
    key_points_left, descriptor_left, key_points_right, descriptor_right = get_keypoint(left_img, right_img)
    good_matches = match_keypoint(key_points_left, key_points_right, descriptor_left, descriptor_right)
    final_homography_matrix = ransac(good_matches)

    rows1, cols1 = right_img.shape[:2]
    rows2, cols2 = left_img.shape[:2]

    points1 = np.float32([[0, 0], [0, rows1], [cols1, rows1], [cols1, 0]]).reshape(-1, 1, 2)
    points = np.float32([[0, 0], [0, rows2], [cols2, rows2], [cols2, 0]]).reshape(-1, 1, 2)
    points2 = cv2.perspectiveTransform(points, final_homography_matrix)
    list_of_points = np.concatenate((points1, points2), axis=0)

    [x_min, y_min] = np.int32(list_of_points.min(axis=0).ravel() - 0.5)
    [x_max, y_max] = np.int32(list_of_points.max(axis=0).ravel() + 0.5)

    translation_matrix = (np.array([[1, 0, (-x_min)], [0, 1, (-y_min)], [0, 0, 1]])).dot(final_homography_matrix)

    output_img = cv2.warpPerspective(left_img, translation_matrix, (x_max - x_min, y_max - y_min))
    output_img[(-y_min):rows1 + (-y_min), (-x_min):cols1 + (-x_min)] = right_img

    return output_img


def get_keypoint(left_img, right_img):
    l_img = cv2.cvtColor(left_img, cv2.COLOR_BGR2GRAY)
    r_img = cv2.cvtColor(right_img, cv2.COLOR_BGR2GRAY)

    surf = cv2.xfeatures2d.SURF_create()
    sift = cv2.xfeatures2d.SIFT_create()

    key_points_left = sift.detect(l_img, None)
    key_points_left, descriptor_left = surf.compute(l_img, key_points_left)

    key_points_right = sift.detect(r_img, None)
    key_points_right, descriptor_right = surf.compute(r_img, key_points_right)
    return key_points_left, descriptor_left, key_points_right, descriptor_right


def match_keypoint(key_points1, key_points2, descriptor1, descriptor2):
    #k-Nearest neighbours between each descriptor
    k_nearest_neighbors = 2
    all_matches = []
    for i, d1 in enumerate(descriptor1):
        dist = []
        for j, d2 in enumerate(descriptor2):
            dist.append([i, j, np.linalg.norm(d1 - d2)])

        dist.sort(key=lambda x: x[2])
        all_matches.append(dist[0:k_nearest_neighbors])

    # Ratio test to get good matches
    good_matches = []

    for m, n in all_matches:
        if m[2] < 0.75 * n[2]:
            left_pt = key_points1[m[0]].pt
            right_pt = key_points2[m[1]].pt
            good_matches.append(
                [left_pt[0], left_pt[1], right_pt[0], right_pt[1]]
            )

    return good_matches


def est_homography_matrix(points):
    coefficient_matrix = []
    for pt in points:
        x, y = pt[0], pt[1]
        X, Y = pt[2], pt[3]
        coefficient_matrix.append([x, y, 1, 0, 0, 0, -1 * X * x, -1 * X * y, -1 * X])
        coefficient_matrix.append([0, 0, 0, x, y, 1, -1 * Y * x, -1 * Y * y, -1 * Y])

    coefficient_matrix = np.array(coefficient_matrix)
    _, _, vh = np.linalg.svd(coefficient_matrix)
    homography_matrix = (vh[-1, :].reshape(3, 3))
    homography_matrix = homography_matrix / homography_matrix[2, 2]
    return homography_matrix


def ransac(good_pts, threshold=5):
    best_inlier_points = []
    final_h_matrix = []

    for _ in range(5000):
        random_pts = random.choices(good_pts, k=4)
        current_h_matrix = est_homography_matrix(random_pts)

        curr_inliers = []

        for pt in good_pts:
            src_pt = np.array([pt[0], pt[1], 1]).reshape(3, 1)
            target_pt = np.array([pt[2], pt[3], 1]).reshape(3, 1)

            transformed_src_pt = np.dot(current_h_matrix, src_pt)
            transformed_src_pt = transformed_src_pt / transformed_src_pt[2]
            dist = np.linalg.norm(target_pt - transformed_src_pt)

            if dist < threshold:
                curr_inliers.append(pt)

        if len(curr_inliers) > len(best_inlier_points):
            best_inlier_points, final_h_matrix = curr_inliers, current_h_matrix
    return final_h_matrix

In [3]:
left_img = cv2.imread('data/left4.jpg')
right_img = cv2.imread('data/right4.jpg')
result_img = stitch_images(left_img, right_img)
cv2.imwrite('results/task4_result.jpg', result_img)

True